In [ ]:
import os
import glob
import datetime

In [ ]:
initial_time = datetime.datetime.now()

os.chdir("/home/working/12S/")
os.getcwd()

In [ ]:
!mkdir trimming

In [ ]:
cd trimming

In [ ]:
files = glob.glob("../../raw_reads/*.fastq.gz")
files = set([f.split("/")[-1].split("_")[0] for f in files])
files = sorted(files)

with open("Sample_accessions.tsv", "w") as sample_accessions:
    sample_accessions.write("SampleID\n" + "\n".join(files)) 

In [ ]:
%%bash

for a in $(cat Sample_accessions.tsv | grep "SampleID" -v)
do
    R1=$(ls -1 ../../raw_reads/$a* | grep "_L001_R1_001.fastq")
    R2=$(ls -1 ../../raw_reads/$a* | grep "_L001_R2_001.fastq")

    echo -e "$a\tfastq\t$R1\t$R2\t18\t18"
done > Querymap.txt

In [ ]:
%%bash

metaBEAT_global.py \
-Q Querymap.txt \
--trim_qual 20 \
--trim_minlength 90 \
--merge \
--product_length 106 \
--read_crop 110 \
--forward_only \
--length_filter 106 \
--length_deviation 0.2 \
-m 12S -n 10 -v \
-@ your_email@gmail.com &> log_trim

In [ ]:
cd ../

In [ ]:
!mkdir chimera_detection

In [ ]:
cd chimera_detection

In [ ]:
%%bash

#Write REFmap
for file in $(ls -1 ../../supplementary_data/reference_DBs/* | grep "reference_database.gb$")
do
      echo -e "$file\tgb"
done >> REFmap.txt

In [ ]:
%%bash

metaBEAT_global.py \
-R REFmap.txt \
-f \
-@ your_email@gmail.com

In [ ]:
%%bash


for a in $(cut -f 1 ../trimming/Querymap.txt)
do
    if [ -s ../trimming/$a/$a\_trimmed.fasta ]
    then
        echo -e "\n### Detecting chimeras in $a ###\n"
        mkdir $a
        cd $a
        vsearch --uchime_ref ../../trimming/$a/$a\_trimmed.fasta --db ../refs.fasta \
        --nonchimeras $a-nonchimeras.fasta --chimeras $a-chimeras.fasta &> log
        cd ..

    else
        echo -e "$a is empty"
    fi
done




In [ ]:
cd ..

In [ ]:
!mkdir non-chimeras

In [ ]:
cd non-chimeras/

In [ ]:
%%bash

#Write REFmap
for file in $(ls -1 ../../supplementary_data/reference_DBs/* | grep "reference_database.gb$")
do
      echo -e "$file\tgb"
done >> REFmap.txt

In [ ]:
%%bash

#Querymap
for a in $(ls -l ../chimera_detection/ | grep "^d" | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
   if [ "$a" != "GLOBAL" ]
   then
      echo -e "$a-nc\tfasta\t../chimera_detection/$a/$a-nonchimeras.fasta"
   fi
done > Querymap.txt

In [ ]:
%%bash

metaBEAT_global.py \
-Q Querymap.txt \
-R REFmap.txt \
--blast --min_ident 1 --min_ali_length 0.85 \
--cluster --clust_match 1 --clust_cov 3 \
-m 12S -n 10 \
-E -v \
-@ your_email@gmail.com \
-o metaBEAT_1.0 &> log_assignment

In [ ]:
final_time = datetime.datetime.now()
total_time = final_time - initial_time

with open("Time.txt", "w") as total_time_file:
    total_time_file.write("Total time to run the pipeline: {TOTAL_TIME}\n".format(
                                TOTAL_TIME=total_time))